<a href="https://colab.research.google.com/github/PiastaS/Deep-Learning-Challenge/blob/main/AlphabetSoupCharityOptimzation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Import Dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import tensorflow as tf
import pandas as pd

In [3]:
#  import and read the .csv file
url = 'https://raw.githubusercontent.com/PiastaS/Deep-Learning-Challenge/main/Resources/Charity_data.csv'
charity_df = pd.read_csv(url)
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,Jan-99,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Drop non-beneficial ID columns
charity_df = charity_df.drop(columns=['EIN', 'NAME', 'STATUS', 'SPECIAL_CONSIDERATIONS'])

In [5]:
# APPLICATION_TYPE value counts for binning
charity_values = charity_df['APPLICATION_TYPE'].value_counts()

# Determine which values to replace if counts are less than 1000
replace_application = charity_values[charity_values < 1000].index.tolist()

# Replace in dataframe
for app in replace_application:
    charity_df.APPLICATION_TYPE = charity_df.APPLICATION_TYPE.replace(app, 'Other')

In [6]:
# APPLICATION_TYPE value counts for binning
charity_values = charity_df['APPLICATION_TYPE'].value_counts()

# Determine which values to replace if counts are less than 1000
replace_application = charity_values[charity_values < 1000].index.tolist()

# Replace in dataframe
for app in replace_application:
    charity_df.APPLICATION_TYPE = charity_df.APPLICATION_TYPE.replace(app, 'Other')

In [7]:
# ASK_AMT bins
ask_values_0 = charity_df['ASK_AMT'][charity_df['ASK_AMT'] == 5000]
ask_values_1 = charity_df['ASK_AMT'][(charity_df['ASK_AMT'] > 5000)]

# Replace in dataframe
for amt in ask_values_0:
    charity_df.ASK_AMT = charity_df.ASK_AMT.replace(amt, '5000')
for amt in ask_values_1:
    charity_df.ASK_AMT = charity_df.ASK_AMT.replace(amt, '>5000')

In [9]:
# AFFILIATION value counts for binning
affiliation_values = charity_df['AFFILIATION'].value_counts()

# Determine which values to replace if counts are less than 1000
replace_affiliation = affiliation_values[affiliation_values < 1000].index.tolist()

# Replace in dataframe
for aff in replace_affiliation:
    charity_df.AFFILIATION = charity_df.AFFILIATION.replace(aff, 'Other')

In [10]:
# ORGANIZATION value counts for binning
organization_values = charity_df['ORGANIZATION'].value_counts()

# Determine which values to replace if counts are less than 1000
replace_organization = organization_values[organization_values < 1000].index.tolist()

# Replace in dataframe
for org in replace_organization:
    charity_df.ORGANIZATION = charity_df.ORGANIZATION.replace(org, 'Other')

In [11]:
# Generate the categorical variable list
application_cat = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()

In [12]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(charity_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [13]:
# Merge one-hot encoded features and drop the originals
charity_df = charity_df.merge(encode_df, left_index=True, right_index=True)
charity_df = charity_df.drop(application_cat, 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
# Split the preprocessed data into features and target arrays
y = charity_df['IS_SUCCESSFUL'].values
X = charity_df.drop(['IS_SUCCESSFUL'], 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 120
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 18

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Third hiZdden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 120)               12000     
                                                                 
 dense_1 (Dense)             (None, 50)                6050      
                                                                 
 dense_2 (Dense)             (None, 18)                918       
                                                                 
 dense_3 (Dense)             (None, 1)                 19        
                                                                 
Total params: 18,987
Trainable params: 18,987
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs('optimized_checkpoints/', exist_ok=True)
checkpoint_path = 'optimized_checkpoints/weights.{epoch:02d}.hdf5'

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4000)

In [19]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=200, callbacks=[cp_callback])

Epoch 1/200
804/804 [==============================] - 2s 2ms/step - loss: 0.6051 - accuracy: 0.6865
Epoch 2/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5737 - accuracy: 0.7224
Epoch 3/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5678 - accuracy: 0.7241
Epoch 4/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5631 - accuracy: 0.7263
Epoch 5/200
779/804 [============================>.] - ETA: 0s - loss: 0.5624 - accuracy: 0.7285
Epoch 5: saving model to optimized_checkpoints/weights.05.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5620 - accuracy: 0.7289
Epoch 6/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5606 - accuracy: 0.7292
Epoch 7/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5599 - accuracy: 0.7293
Epoch 8/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5584 - accuracy: 0.7299
Epoch 9/200
804/804 [=======================

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

268/268 - 0s - loss: 1.0973 - accuracy: 0.6645 - 447ms/epoch - 2ms/step
Loss: 1.0972903966903687, Accuracy: 0.6644898056983948


In [22]:
# Export the model to HDF5 file
nn.save('AlphabetSoupCharityOptimization.h5')